# Exploring Museum Data

**Enter Imports**

In [14]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from os import environ as ENV

Create a connection.

In [16]:
load_dotenv()

conn = psycopg2.connect(
    dbname=ENV['DB_NAME'],
        user=ENV['DB_USER'],
        password=ENV['DB_PASSWORD'],
        host=ENV['DB_HOST'],
        port=ENV['DB_PORT'])


### Highest Average Rating Per Exhibit.

In [19]:
cur = conn.cursor()
result = cur.execute("""SELECT exhibition_name, avg(rating_id) avg_rating
                        FROM exhibition
                        JOIN review USING(exhibition_id)
                        GROUP BY exhibition_name
                        ORDER BY avg_rating DESC;""")
result = cur.fetchall()
cur.close()

df =  pd.DataFrame(result, columns=['exhibition_name', 'avg_rating'])
print(df)

           exhibition_name          avg_rating
0  The Crenshaw Collection  2.8305084745762712
1       Measureless to Man  1.9294117647058824
2       Our Polluted World  1.9169435215946844
3               Adaptation  1.4259740259740260
4      Cetacean Sensations  1.2162162162162162


The exhibition with the best rating is The Crenshaw Collection. Significantly better than second. The worst rated was Cetacean Sensations.

### Look at total engagement

In [20]:
cur = conn.cursor()
result = cur.execute("""SELECT exhibition_name, COUNT(review_id) AS total_ratings
                        FROM exhibition
                        JOIN review USING(exhibition_id)
                        GROUP BY exhibition_name
                        ORDER BY total_ratings DESC;""")
result = cur.fetchall()
cur.close()

df = pd.DataFrame(result, columns=['exhibition_name', 'total_ratings'])
print(df)

           exhibition_name  total_ratings
0      Cetacean Sensations            481
1  The Crenshaw Collection            472
2       Measureless to Man            425
3               Adaptation            385
4       Our Polluted World            301


While Cetacean Sensations was the lowest rating it caused the most engagement. Likely meaning people were disappointed enough to leave a review. Our Polluted World may have been forgettable and the viewer didn't want to leave feedback.

### Look at the average rating per floor.

In [21]:
cur = conn.cursor()
result = cur.execute("""SELECT floor_name, avg(rating_id) AS avg_rating
                        FROM floor_assignment
                        JOIN exhibition USING(exhibition_id)
                        JOIN floor USING(floor_id)
                        JOIN review USING(exhibition_id)
                        GROUP BY floor_name
                        ORDER BY avg_rating DESC;""")
result = cur.fetchall()
cur.close()

df = pd.DataFrame(result, columns=['floor_name', 'avg_rating'])
print(df)

  floor_name          avg_rating
0          2  2.8305084745762712
1          3  1.9169435215946844
2          1  1.5507726269315673
3      Vault  1.4259740259740260


Perhaps the Vault meant it was a worse environment with less windows? Not enough exhibitions to read too much into the data here.